In [54]:
import numpy as np
from collections import Counter, defaultdict, namedtuple, deque
from math import gcd, ceil
import re
import networkx as nx
from dataclasses import dataclass
import itertools
import aoc
from matplotlib import pyplot as plt
import string
# plt.imshow(pic)

'97: 138 57 | 12 83'

'babbaaaabbbbbbabaaaaabbb'

In [63]:
f=open('rules.txt')
f=open('rulestest.txt')
f=open('oldrules.txt')
lines = [line.rstrip('\n') for line in f]
rules = {}
for line in lines:
    num, ins = line.split(': ')
    ins = ins.split('|')
    rules[num] = [i.split() for i in ins]


f=open('messages.txt')
# f=open('messagestest.txt')
mess = [line.rstrip('\n') for line in f]


def getoptions(key):
    ins = rules[key]
    alloptions = []
    for option in ins: # go through all the options, can be 1 if 1 option. this can still yield multiple results through recursion
        # print('option', option)
        ans = []
        for i in option:
            if i.isnumeric():
                res = getoptions(i)
                ans.append(res)
            else:
                ans.append(i) # i is a or b

        temp = [''.join(comb) for comb in itertools.product(*ans)]
            # if len(temp)==1:
            #     alloptions.append(temp[0])
            # else:
        alloptions.append(temp)
    if len(alloptions)==1:
        return alloptions[0]
    else:
        # print(aoc.flatten(alloptions))
        return aoc.flatten(alloptions)


128

In [46]:
ins = getoptions('0')
ans =0
for line in mess:
    if line in ins: ans +=1
ans

129

In [50]:
# f=open('rulestest.txt')
lines = [line.rstrip('\n') for line in f]
rules = {}
for line in lines:
    num, ins = line.split(': ')
    ins = ins.split('|')
    rules[num] = [i.split() for i in ins]
getoptions('0')


KeyError: '0'

In [70]:
forty = getoptions('42')
thirty = getoptions('31')
8 + 11
42 + optioneelzichzelf
11: 42 31 | 42 11 31

42 + (optioneel 42) + 42 (optioneel combo 42 en 31) 31


In [77]:
cut = len(max((m for m in mess), key = len))
combinations = cut/8
combinations
def gen_forty():
    for i in range(1,combinations+1)
    len = 1

    

12.0

In [152]:
m = mess[0]
ans = 0
nok = 0

for m in mess:
    
    cut = len(m)//8
    # print('new ', cut)
    darkmode = False
    nextforty = True
    assert cut * 8 == len(m)
    for i in range(cut):
        # print(i)
        # print(m[i*8:i*8+8])
        temp = m[i*8:i*8+8]
        # print(len(temp))
        if (i == 0 and temp not in forty) or (i == 1 and temp not in forty):
            # print('begin nok')
            nok+=1
            break
        elif i > 1:
            if (i == cut-1):
                if temp in thirty:
                    ans+=1
                # print('valid')
                    break
                else:
                    nok+=1
                    break
            if not darkmode:
                if temp in forty:
                    continue
                if temp in thirty:
                    print('darkmode')
                    darkmode = True
                    nextforty = True
                    continue
            elif darkmode:
                print('darkmode')
                if temp in forty and nextforty and i != cut -2:
                    nextforty = False
                    continue
                elif temp in thirty and not nextforty:
                    nextforty = True
                    continue
                else:
                    print('nok')
                    nok+=1
                    break
            else:
                # nok+=1
                break
ans

darkmode
darkmode
nok
darkmode
darkmode
nok
darkmode
darkmode
nok
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
nok
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
nok
darkmode
darkmode
darkmode
nok
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode
nok
darkmode
darkmode
darkmode
darkmode
darkmode
darkmode

243

In [153]:
ans

243

In [147]:
267+89

356

In [66]:
aoc.flatten([['aa'],['bb'],[['cc','dd']]])

['aa', 'bb', 'cc', 'dd']

In [67]:
def is_single_list(item):
    if isinstance(item, list):
        for i in item:
            if isinstance(i, list):
                return False
        else:
            return True
    else:
        return True




In [68]:
'2' in {'22'}

False

In [69]:
def getoptions(key):
    ins = rules[key]
    alloptions = []
    for option in ins: # go through all the options, can be 1 if 1 option. this can still yield multiple results through recursion
        # print('option', option)
        ans = []
        for i in option:
            if i.isnumeric():
                res = getoptions(i)
                ans.append(res)
            else:
                ans.append(i) # i is a or b

        temp = [''.join(comb) for comb in itertools.product(*ans)]
            # if len(temp)==1:
            #     alloptions.append(temp[0])
            # else:
        alloptions.append(temp)
    if len(alloptions)==1:
        return alloptions[0]
    else:
        # print(aoc.flatten(alloptions))
        return aoc.flatten(alloptions)
# getoptions('0')